<a href="https://colab.research.google.com/github/gaelle-cohen/dotfiles/blob/master/Yelp_reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [78]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras 
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping
from keras.preprocessing.text import Tokenizer
import gensim
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import train_test_split


In [63]:
# pip install wandb
import wandb
from wandb.keras import WandbCallback
wandb.init(project="le-wagon-challenge")

W&B Run: https://app.wandb.ai/gaelle-cohen/le-wagon-challenge/runs/50acmmy2

In [69]:
!pip install wandb

In [2]:
## PREPROCESSING
# import data sets
# splits into sentences
# word2vec

In [20]:
# Import dataset test_reviews
X_test = pd.read_csv('test_reviews.csv')
X_test

,review
0,So disappointing we went there to waste time b...
1,"Nice atmosphere, good quality and fast service..."
2,This is my go to spot to get supplements! They...
3,Ottima la cipolla fritta tagliata solamente me...
4,"Enjoyed Guy's show immensely, unfortunately th..."
...,...
19995,"A little pricey, but it's good and for what it..."
19996,I'm a huge fan of Korean food so when I saw th...
19997,As far as we are concerned this is the best An...
19998,These guys really know their stuff!! My Dodge...


In [19]:
(X_test.shape)

(20000, 1)

In [32]:
# Split into sentences

sentences_test = []
for x in range(X_test.shape[0]):
    sentences_test.append(X_test['review'].iloc[x].split())
    
#n sentences_test

In [22]:
#X_test['review'][1].split()

In [23]:
# Import dataset train_reviews
X_train = pd.read_csv('train_reviews.csv')
X_train

,review,star
0,Total bill for this horrible service? Over $8G...,1.0
1,I *adore* Travis at the Hard Rock's new Kelly ...,5.0
2,I have to say that this office really has it t...,5.0
3,Went in for a lunch. Steak sandwich was delici...,5.0
4,Today was my second out of three sessions I ha...,1.0
...,...,...
99996,We went here to have a small dinner with famil...,1.0
99997,Amazing!!! Our 2 boys never had this much fun ...,5.0
99998,Wanted a short hair cut but didn't have a regu...,5.0
99999,"This place is rad! Dueling pianos, happy hour ...",5.0


In [43]:
sentences_train = []
for x in range(X_train.shape[0]):
    sentences_train.append(X_train['review'].iloc[x].split())
    
sentences_train

[['Total',
  'bill',
  'for',
  'this',
  'horrible',
  'service?',
  'Over',
  '$8Gs.',
  'These',
  'crooks',
  'actually',
  'had',
  'the',
  'nerve',
  'to',
  'charge',
  'us',
  '$69',
  'for',
  '3',
  'pills.',
  'I',
  'checked',
  'online',
  'the',
  'pills',
  'can',
  'be',
  'had',
  'for',
  '19',
  'cents',
  'EACH!',
  'Avoid',
  'Hospital',
  'ERs',
  'at',
  'all',
  'costs.'],
 ['I',
  '*adore*',
  'Travis',
  'at',
  'the',
  'Hard',
  "Rock's",
  'new',
  'Kelly',
  'Cardenas',
  'Salon!',
  "I'm",
  'always',
  'a',
  'fan',
  'of',
  'a',
  'great',
  'blowout',
  'and',
  'no',
  'stranger',
  'to',
  'the',
  'chains',
  'that',
  'offer',
  'this',
  'service;',
  'however,',
  'Travis',
  'has',
  'taken',
  'the',
  'flawless',
  'blowout',
  'to',
  'a',
  'whole',
  'new',
  'level!',
  "Travis's",
  'greets',
  'you',
  'with',
  'his',
  'perfectly',
  'green',
  'swoosh',
  'in',
  'his',
  'otherwise',
  'perfectly',
  'styled',
  'black',
  'hair',


In [25]:
y_train = np.array(X_train['star'])
y_train # In deep learning, y_train always need to be an array

array([1., 5., 5., ..., 5., 5., 5.])

In [ ]:
## Word embedding with Word2Vec

In [26]:
class MyCorpus(object):
    """An interator that yields sentences (lists of str)."""

    def __iter__(self):
        for line in sentences_train:
            # assume there's one document per line, tokens separated by whitespace
            yield line
            
corpus = MyCorpus()


In [31]:
corpus

In [27]:
model = gensim.models.Word2Vec(sentences=corpus)

In [50]:
#print(model.wv.vocab)

In [38]:
model.wv.most_similar('king')

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('legs,', 0.7084224224090576),
 ('Alaskan', 0.7031800746917725),
 ('jumbo', 0.692030131816864),
 ('lump', 0.6750541925430298),
 ('mini', 0.6628178358078003),
 ('snow', 0.6595959663391113),
 ('crackers', 0.6533161401748657),
 ('clams,', 0.6396863460540771),
 ('crab', 0.6267260313034058),
 ('legs.', 0.6219942569732666)]

In [58]:
def compute_mean_embedding(l):
    """
    l: list of words
    """
    liste = []
    for word in l:
        if word in model.wv.vocab:
            liste.append(model.wv[word])
    if len(liste) != 0:
        return (True, np.mean(np.array(liste), axis = 0))
    else:
        return (False, False)       


In [72]:
y_train_filter = []


mean_vectors = []
for i, sentence in enumerate(sentences_train):
    tmp = compute_mean_embedding(sentence)
    if tmp[0]:
        mean_vectors.append(tmp[1])
        y_train_filter.append(y_train[i])

In [65]:
np.array(mean_vectors).shape

(99982, 100)

In [73]:
len(y_train_filter)

99982

In [76]:
#Initialiser un modele de regression logistique et onevsrest classifier

ovr_log = OneVsRestClassifier(LogisticRegression(max_iter = 200))
ovr_log.fit(mean_vectors, y_train_filter)

OneVsRestClassifier(estimator=LogisticRegression(C=1.0, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=200,
                                                 multi_class='auto',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=None,
                                                 solver='lbfgs', tol=0.0001,
                                                 verbose=0, warm_start=False),
                    n_jobs=None)

In [75]:
ovr_log.score(mean_vectors,y_train_filter)

0.6173611250025004